In [1]:
import os
import cv2
import collections
import time
import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold, train_test_split

import torchvision
import torchvision.transforms as transforms
import torch
from torch.utils.data import TensorDataset, DataLoader,Dataset
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau, CosineAnnealingLR

import albumentations as albu
import configparser
import argparse
import wandb

# Catalyst is amazing.
from catalyst.data import Augmentor
from catalyst.dl import utils
from catalyst.data.reader import ImageReader, ScalarReader, ReaderCompose, LambdaReader
# from catalyst.dl.runner import SupervisedRunner
from catalyst.dl.runner import SupervisedWandbRunner as SupervisedRunner
from catalyst.contrib.models.segmentation import Unet
from catalyst.dl.callbacks import DiceCallback, EarlyStoppingCallback, InferCallback, CheckpointCallback

# PyTorch made my work much much easier.
import segmentation_models_pytorch as smp
from dataloader import SegmentationDataset, SegmentationDatasetTest
from augmentations import get_training_augmentation, get_validation_augmentation, get_preprocessing

In [2]:
device=torch.device('cuda')

def get_ids():
    train = pd.read_csv(f'../input/train.csv')
    train['label'] = train['Image_Label'].apply(lambda x: x.split('_')[1])
    train['im_id'] = train['Image_Label'].apply(lambda x: x.split('_')[0])
    id_mask_count = train.loc[train['EncodedPixels'].isnull() == False, 'Image_Label'].apply(lambda x: x.split('_')[0]).value_counts().\
    reset_index().rename(columns={'index': 'img_id', 'Image_Label': 'count'})
    train_ids, valid_ids = train_test_split(id_mask_count['img_id'].values, random_state=42, stratify=id_mask_count['count'], test_size=0.1)
    return train_ids, valid_ids

def get_loaders(bs=32, num_workers=4, preprocessing_fn=None):
        train_dataset = SegmentationDataset(ids=train_ids,
                    transforms=get_training_augmentation(),
                    preprocessing=get_preprocessing(preprocessing_fn),
                    img_db="../input/train_images_525/train_images_525",
                    mask_db="../input/mask")
        
        valid_dataset = SegmentationDataset(ids=valid_ids,
                    transforms=get_validation_augmentation(),
                    preprocessing=get_preprocessing(preprocessing_fn),
                    img_db="../input/train_images_525/train_images_525",
                    mask_db="../input/mask")

        train_loader = DataLoader(train_dataset, batch_size=bs,
            shuffle=True, num_workers=num_workers)
        valid_loader = DataLoader(valid_dataset, batch_size=bs,
            shuffle=False, num_workers=num_workers)

        loaders = {
            "train": train_loader,
            "valid": valid_loader
        }
        return loaders

def get_model(encoder='resnet18', classes=4):
    encoder_weights = 'imagenet'
    model = smp.Unet(
        encoder_name=encoder,
        encoder_weights=encoder_weights,
        classes=classes,
        activation=None,
    )
    preprocessing_fn = smp.encoders.get_preprocessing_fn(encoder, encoder_weights)
    return model, preprocessing_fn

In [3]:
config = configparser.ConfigParser()
config.read('../configs/config.ini')
conf = config['DEFAULT']

lrd = conf.getfloat('lrd')
lre = conf.getfloat('lre')
epochs = conf.getint('epochs')
num_workers = conf.getint('num_workers')
encoder = conf.get('encoder')
logdir = conf.get('logdir')
bs = 2
train_ids, valid_ids = get_ids()
model, preprocessing_fn = get_model(encoder)
loaders = get_loaders(bs, num_workers, preprocessing_fn)

optimizer = torch.optim.Adam([
        {'params': model.decoder.parameters(), 'lr': lrd},
        {'params': model.encoder.parameters(), 'lr': lre},
    ])

/home/timetraveller/.local/lib/python3.7/site-packages/albumentations/augmentations/transforms.py:2567: UserWarning:

Using lambda is incompatible with multiprocessing. Consider using regular functions or partial().



In [4]:
def dice(pr, gt, eps=1e-9, threshold=None, activation='sigmoid'):
    if activation is None or activation == "none":
        activation_fn = lambda x: x
    elif activation == "sigmoid":
        activation_fn = torch.nn.Sigmoid()
    elif activation == "softmax2d":
        activation_fn = torch.nn.Softmax2d()
    else:
        raise NotImplementedError(
            "Activation implemented for sigmoid and softmax2d"
        )

    pr = activation_fn(pr)

    if threshold is not None:
        pr = (pr > threshold).float()


    intersection = torch.sum(gt * pr)
    sum1 = torch.sum(pr)
    sum2 = torch.sum(gt)

    score = (2.0 * intersection + eps) / (sum1 + sum2 + eps)


    return score

class DiceLoss(nn.Module):
    __name__ = 'dice_loss'

    def __init__(self, eps=1e-9, activation='sigmoid'):
        super().__init__()
        self.activation = activation
        self.eps = eps

    def forward(self, y_pr, y_gt):
        return 1 - dice(y_pr, y_gt, eps=self.eps,
                    threshold=None, activation=self.activation)


class BCEDiceLoss(DiceLoss):
    __name__ = 'bce_dice_loss'

    def __init__(self, eps=1e-9, activation='sigmoid'):
        super().__init__(eps, activation)
        self.bce = nn.BCEWithLogitsLoss()

    def forward(self, y_pr, y_gt):
        dice = super().forward(y_pr, y_gt)
        bce = self.bce(y_pr, y_gt)
        return 0.5*dice + 0.5*bce

In [5]:
criteria = BCEDiceLoss()

for batch in loaders['train']:
    print(len(batch))
    img, mask = batch
    output = model(img)
    loss = criteria(output, mask)
    print(loss)
    print(output.shape)
    print(img.shape)
    print(mask.shape)
    break

2
tensor(0.7413, grad_fn=<AddBackward0>)
torch.Size([2, 4, 320, 640])
torch.Size([2, 3, 320, 640])
torch.Size([2, 4, 320, 640])


In [38]:
eps = 1
gt, pr = mask, output
activation_fn = torch.nn.Sigmoid()
pr = activation_fn(pr)
intersection = torch.sum(gt * pr)
sum1 = torch.sum(pr)
sum2 = torch.sum(gt)
score = (2.0 * intersection + eps) / (sum1 + sum2 + eps)
print(score)

tensor(0.2264, grad_fn=<DivBackward0>)


In [45]:
eps = 1e-7
gt, pr = mask, output
activation_fn = torch.nn.Sigmoid()
pr = activation_fn(pr)
intersection = torch.sum(gt * pr, axis=[0, 2, 3])
sum1 = torch.sum(pr, axis=[0, 2, 3])
sum2 = torch.sum(gt, axis=[0, 2, 3])
score = (2.0 * intersection + eps) / (sum1 + sum2 + eps)
print(torch.mean(score))

tensor(0.1877, grad_fn=<MeanBackward0>)
